<a href="https://colab.research.google.com/github/KevFerris/Real-time-tides/blob/main/NS_tides.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install requests
!pip install python-dateutil
!pip install folium
!pip install pytz


In [2]:
import requests
import datetime
import dateutil.parser
import time

# Function to round time to the nearest half hour
def round_time_to_nearest_half_hour(dt):
    minute = 30 if dt.minute >= 15 and dt.minute < 45 else 0
    return dt.replace(minute=minute, second=0, microsecond=0)

# Function to find closest half-hour data
def find_closest_half_hour(data_list):
    target_minute = 30 if dateutil.parser.parse(data_list[0]['eventDate']).minute < 30 else 0
    closest_data = None
    min_difference = float('inf')

    for data in data_list:
        event_time = dateutil.parser.parse(data['eventDate'])
        minute_diff = abs(event_time.minute - target_minute)

        if minute_diff < min_difference:
            min_difference = minute_diff
            closest_data = data

    return closest_data

# Function to fetch data for a given station ID
def fetch_station_data(station_id, from_date, to_date, time_series_code="wlp"):
    url = f"https://api-iwls.dfo-mpo.gc.ca/api/v1/stations/{station_id}/data?time-series-code={time_series_code}&from={from_date}&to={to_date}"
    response = requests.get(url)
    time.sleep(0.1)
    if response.status_code == 200:
        return response.json()
    else:
        print(f"Failed to retrieve data for station {station_id} - Status Code: {response.status_code}")
        return None

# Calculate the nearest half hour from the current time
current_time = datetime.datetime.utcnow()
rounded_time = round_time_to_nearest_half_hour(current_time)
# Format times for the API request
from_date = (rounded_time - datetime.timedelta(minutes=30)).isoformat() + 'Z'
to_date = rounded_time.isoformat() + 'Z'

# Station IDs and their names
stations = {
    "5cebf1df3d0f4a073c4bbcbb": "Halifax",
    "5cebf1e33d0f4a073c4bc22f": "Pugwash",
    "5cebf1df3d0f4a073c4bbc84": "Meteghan",
    "5cebf1df3d0f4a073c4bbcaf": "Liverpool",
    "5cebf1df3d0f4a073c4bbcd1": "Port Bickerton",
    "5cebf1e13d0f4a073c4bbef9": "Cheticamp",
    "5cebf1e03d0f4a073c4bbe9d": "Port Dufferin",
    "5cebf1df3d0f4a073c4bbc30": "Herring Cove",
    "5cebf1df3d0f4a073c4bbc6a": "Parkers Cove",
    "5cebf1e33d0f4a073c4bc209": "Pictou",
    #"5cebf1e23d0f4a073c4bbfb4": "Truro",
    #"5dd5aa6aff231600015c1a03": "Maitland",
    "5cebf1df3d0f4a073c4bbc58": "Burntcoat Head",
    "5cebf1e23d0f4a073c4bc10a": "Sand Point",
    "5cebf1df3d0f4a073c4bbc68": "Margaretsville",
    "5cebf1e33d0f4a073c4bc203": "Ballantynes Cove",
    "5cebf1e33d0f4a073c4bc1ed": "Bay St. Lawrence",
    "5cebf1e03d0f4a073c4bbe97": "Prospect",
    "5cebf1e23d0f4a073c4bc11a": "Glace Bay",
    "5cebf1e33d0f4a073c4bc1ff": "Cape Jack",
    "5cebf1e33d0f4a073c4bc12b": "St Anns Harbour",
    "5cebf1e03d0f4a073c4bbe8d": "Riverport",
    "5cebf1e03d0f4a073c4bbea8": "Fourchu",
    "5cebf1df3d0f4a073c4bbc44": "Joggins",
    "5cebf1e33d0f4a073c4bc231": "Tidnish",
    "5cebf1e33d0f4a073c4bc12d": "Ingonish Beach",
    "5cebf1df3d0f4a073c4bbcc1": "Murphys Cove",
    "5cebf1df3d0f4a073c4bbca7": "Upper Port la Tour",
    "5cebf1df3d0f4a073c4bbc4a": "Advocate",
    "5cebf1e33d0f4a073c4bc1f9": "Broad Cove Marsh",
    "5cebf1df3d0f4a073c4bbc74": "Sandy Cove",
    "5cebf1df3d0f4a073c4bbc60": "Cape Blomidon",
    "5cebf1e33d0f4a073c4bc22d": "Cape Cliff",
    "5cebf1df3d0f4a073c4bbc6c": "DIGBY",
    "5cebf1e03d0f4a073c4bbe83": "Wedgeport",
    "5cebf1df3d0f4a073c4bbcdb": "Canso Harbour",
    #"5cebf1e13d0f4a073c4bbeae": "Johnstown",
    "5cebf1e23d0f4a073c4bc123": "North Sydney",
    "5dd30650e0fdc4b9b4be6b71": "Iona",
    "5cebf1df3d0f4a073c4bbcd9": "Sable Island",
    "5cebf1e33d0f4a073c4bc1fb": "Port Hood",
    "5cebf1e33d0f4a073c4bc1f5": "Margaree Harbour",
    "5cebf1e33d0f4a073c4bc229": "Skinners Cove",
    "5cebf1df3d0f4a073c4bbc56": "Five Islands",
    "5cebf1df3d0f4a073c4bbc5e": "Hantsport",
    "5cebf1df3d0f4a073c4bbcab": "Lockeport",
    "5cebf1df3d0f4a073c4bbc86": "Port Maitland",
    "5cebf1e23d0f4a073c4bc112": "Petit De Grat",
    "5cebf1e23d0f4a073c4bc118": "Louisbourg",
    #"5cebf1df3d0f4a073c4bbc42": "Sackville",
    "5cebf1df3d0f4a073c4bbc36": "Hopewell Cape",
    "5dd30650e0fdc4b9b4be6d40": "Chester",
    #"5cebf1de3d0f4a073c4bb90f": "Guysborough",
    "5cebf1df3d0f4a073c4bbcd5": "Larrys River"
}


# Fetch and process data for each station
for station_id, station_name in stations.items():
    data = fetch_station_data(station_id, from_date, to_date)
    if data:
        closest_data = find_closest_half_hour(data)
        if closest_data:
            #print(f"{station_name}")
            #print(closest_data)
            this = 'doesnothing'


Failed to retrieve data for station 5cebf1df3d0f4a073c4bbcbb - Status Code: 404


In [3]:
import csv

station_coordinates = {
    "5cebf1df3d0f4a073c4bbcbb": {"lat": 44.666667, "lon": -63.583333, "CD_offset": -1.445821762},  # Halifax
    "5cebf1e33d0f4a073c4bc22f": {"lat": 45.85,"lon": -63.6833,"CD_offset": -2.138676166}, # Pugwash
    "5cebf1df3d0f4a073c4bbc84": {"lat": 44.2, "lon": -66.166667, "CD_offset": -3.536155224},  # Meteghan
    "5cebf1df3d0f4a073c4bbcaf": {"lat": 44.05, "lon": -64.716667, "CD_offset": -1.740411043},  # Liverpool
    "5cebf1df3d0f4a073c4bbcd1": {"lat": 45.1, "lon": -61.733333, "CD_offset": -1.59133625},  # Port Bickerton
    "5cebf1e13d0f4a073c4bbef9": {"lat": 46.633333, "lon": -61.016667, "CD_offset": -1.23145473},  # Cheticamp
    "5cebf1e03d0f4a073c4bbe9d": {"lat": 44.9, "lon": -62.383333, "CD_offset": -1.692808747},  # Port Dufferin
    "5cebf1df3d0f4a073c4bbc30": {"lat": 45.566667, "lon": -64.966667, "CD_offset": -6.694088459},  # Herring Cove
    "5cebf1df3d0f4a073c4bbc6a": {"lat": 44.8, "lon": -65.533333, "CD_offset": -5.345516205},  # Parkers Cove
    "5cebf1e33d0f4a073c4bc209": {"lat": 45.683333, "lon": -62.7, "CD_offset": -1.563264012},  # Pictou
    "5cebf1df3d0f4a073c4bbc58": {"lat": 45.3, "lon": -63.8, "CD_offset": -7.734239101},  # Burntcoat Head
    "5cebf1e23d0f4a073c4bc10a": {"lat": 45.516667, "lon": -61.266667, "CD_offset": -1.32016933},  # Sand Point
    "5cebf1df3d0f4a073c4bbc68": {"lat": 45.05, "lon": -65.066667, "CD_offset": -6.05305624},  # Margaretsville
    "5cebf1e33d0f4a073c4bc203": {"lat": 45.858, "lon": -61.918, "CD_offset": -1.324637771},  # Ballantynes Cove
    "5cebf1e33d0f4a073c4bc1ed": {"lat": 47.016667, "lon": -60.45, "CD_offset": -1.07204175},  # Bay St. Lawrence
    "5cebf1e03d0f4a073c4bbe97": {"lat": 44.45, "lon": -63.783333, "CD_offset": -1.459552884},  # Prospect
    "5cebf1e33d0f4a073c4bc1ff": {"lat": 45.7, "lon": -61.55, "CD_offset": -1.347409248},  # Cape Jack
    "5cebf1e03d0f4a073c4bbe8d": {"lat": 44.283333, "lon": -64.35, "CD_offset": -1.625600576},  # Riverport
    "5cebf1e03d0f4a073c4bbea8": {"lat": 45.716667, "lon": -60.25, "CD_offset": -1.514823318},  # Fourchu
    "5cebf1df3d0f4a073c4bbc44": {"lat": 45.683333, "lon": -64.466667, "CD_offset": -7.331525326},  # Joggins
    "5cebf1e33d0f4a073c4bc231": {"lat": 46, "lon": -64.016667, "CD_offset": -2.29026413},  # Tidnish
    "5cebf1e33d0f4a073c4bc12d": {"lat": 46.633333, "lon": -60.383333, "CD_offset": -1.101977348},  # Ingonish Beach
    "5cebf1df3d0f4a073c4bbcc1": {"lat": 44.783333, "lon": -62.783333, "CD_offset": -1.735851169},  # Murphys Cove
    "5cebf1df3d0f4a073c4bbca7": {"lat": 43.516667, "lon": -65.466667, "CD_offset": -1.798464179},  # Upper Port la Tour
    "5cebf1df3d0f4a073c4bbc4a": {"lat": 45.333333, "lon": -64.783333, "CD_offset": -6.310697556},  # Advocate
    "5cebf1e33d0f4a073c4bc1f9": {"lat": 46.3, "lon": -61.266667, "CD_offset": -1.128771901},  # Broad Cove Marsh
    "5cebf1df3d0f4a073c4bbc74": {"lat": 44.5, "lon": -66.1, "CD_offset": -4.146985054},  # Sandy Cove
    "5cebf1df3d0f4a073c4bbc60": {"lat": 45.266667, "lon": -64.35, "CD_offset": -7.427913189},  # Cape Blomidon
    "5cebf1e33d0f4a073c4bc22d": {"lat": 45.883333, "lon": -63.433333, "CD_offset": -1.949799299},  # Cape Cliff
    "5cebf1df3d0f4a073c4bbc6c": {"lat": 44.633333, "lon": -65.75, "CD_offset": -4.949921131},  # DIGBY
    "5cebf1e03d0f4a073c4bbe83": {"lat": 43.716667, "lon": -65.983333, "CD_offset": -2.628114462},  # Wedgeport
    "5cebf1df3d0f4a073c4bbcdb": {"lat": 45.333333, "lon": -61, "CD_offset": -1.323019981},  # Canso Harbour
    "5cebf1e23d0f4a073c4bc123": {"lat": 46.20879, "lon": -60.24519, "CD_offset": -1.073629022},  # North Sydney
    "5dd30650e0fdc4b9b4be6b71": {"lat": 45.9567, "lon": -60.79396, "CD_offset": -0.667317271},  # Iona
    "5cebf1df3d0f4a073c4bbcd9": {"lat": 43.9666, "lon": -59.7999, "CD_offset": -1.342890501},  # Sable Island
    "5cebf1e33d0f4a073c4bc1fb": {"lat": 46.0279, "lon": -61.545, "CD_offset": -1.08808744},  # Port Hood
    "5cebf1e33d0f4a073c4bc1f5": {"lat": 46.4333, "lon": -61.1, "CD_offset": -1.034189582},  # Margaree Harbour
    "5cebf1e33d0f4a073c4bc229": {"lat": 45.7999, "lon": -63.0499, "CD_offset": -2.064927816},  # Skinners Cove
    "5cebf1df3d0f4a073c4bbc56": {"lat": 45.3833, "lon": -64.1333, "CD_offset": -7.824222565},  # Five Islands
    "5cebf1df3d0f4a073c4bbc5e": {"lat": 45.0666, "lon": -64.1666, "CD_offset": -7.638429165},  # Hantsport
    "5cebf1df3d0f4a073c4bbcab": {"lat": 43.7, "lon": -65.1166, "CD_offset": -1.934431791},  # Lockeport
    "5cebf1df3d0f4a073c4bbc86": {"lat": 43.9833, "lon": -66.15, "CD_offset": -3.098487616},  # Port Maitland
    "5cebf1e23d0f4a073c4bc112": {"lat": 45.5, "lon": -60.9666, "CD_offset": -1.560625196},  # Petit De Grat
    "5cebf1e23d0f4a073c4bc118": {"lat": 45.9166, "lon": -59.9666, "CD_offset": -1.500151515},  # Louisbourg
    "5cebf1df3d0f4a073c4bbc36": {"lat": 45.85, "lon": -64.5833, "CD_offset": -7.534975529},  # Hopewell Cape
    "5dd30650e0fdc4b9b4be6d40": {"lat": 44.5333, "lon": -64.2333, "CD_offset": -1.58852303},  # Chester
    "5cebf1df3d0f4a073c4bbcd5": {"lat": 45.2166, "lon": -61.3833, "CD_offset": -1.468343139}  # Larrys River
}

# Write data to CSV including the CGVD13 value
with open('tide_data.csv', 'w', newline='') as file:
    writer = csv.writer(file)
    writer.writerow(['Station Name', 'Station ID', 'Event Date', 'Value', 'CGVD13 Value', 'Latitude', 'Longitude'])

    for station_id, station_info in station_coordinates.items():
        data = fetch_station_data(station_id, from_date, to_date)
        if data:
            closest_data = find_closest_half_hour(data)
            if closest_data:
                # Calculate the CGVD13 value
                cgvd13_value = round(closest_data['value'] + station_info["CD_offset"], 2)
                # Write data to CSV file
                writer.writerow([
                    stations[station_id],  # Station Name
                    station_id,  # Station ID
                    closest_data['eventDate'],  # Event Date
                    closest_data['value'],  # Value
                    cgvd13_value,  # CGVD13 Value
                    station_info["lat"],  # Latitude
                    station_info["lon"]  # Longitude
                ])

Failed to retrieve data for station 5cebf1df3d0f4a073c4bbcbb - Status Code: 404


In [4]:
import pandas as pd

# Load the CSV file into a DataFrame
df_tide = pd.read_csv('tide_data.csv')

# Display the DataFrame
#df_tide

In [5]:
import requests
import datetime
import dateutil.parser
from dateutil.parser import parse
import csv
import pytz
from dateutil.tz import tzutc
from dateutil.tz import UTC

def fetch_station_data(station_id, from_date, to_date, time_series_code="wlp-hilo"):
    url = f"https://api-iwls.dfo-mpo.gc.ca/api/v1/stations/{station_id}/data?time-series-code={time_series_code}&from={from_date}&to={to_date}"
    response = requests.get(url)
    #print(url)
    if response.status_code == 200:
        return response.json()
    else:
        print(f"Failed to retrieve data for station {station_id} - Status Code: {response.status_code}")
        return None

# Calculate the time range
current_time = datetime.datetime.utcnow()
from_date = (current_time - datetime.timedelta(hours=12)).isoformat() + 'Z'
to_date = (current_time + datetime.timedelta(hours=16)).isoformat() + 'Z'

all_stations_tide_data = {}

# Fetch and store data for each station
for station_id, station_name in stations.items():
    data = fetch_station_data(station_id, from_date, to_date)
    time.sleep(0.1)
    if data:
        all_stations_tide_data[station_id] = {
            "station_name": station_name,
            "tide_events": data
        }

# Your existing code for fetching data and initializing `all_stations_tide_data` goes here
tide_data = {}
for station_id, data in all_stations_tide_data.items():
    sorted_tide_events = sorted(data['tide_events'], key=lambda x: dateutil.parser.parse(x['eventDate']))

    last_tide = None
    high_tides = []
    low_tides = []

    # Process each event to classify them and find the last tide
    for event in sorted_tide_events:
        current_time = current_time.replace(tzinfo=UTC)
        event_date = dateutil.parser.parse(event['eventDate'])

        # Determine the last tide event
        if event_date < current_time:
            last_tide = event

        # Classify as high or low tide
        if len(sorted_tide_events) > 1:
            next_index = sorted_tide_events.index(event) + 1
            if next_index < len(sorted_tide_events):
                next_event = sorted_tide_events[next_index]
                if event['value'] < next_event['value']:
                    low_tides.append(event)
                else:
                    high_tides.append(event)

    # Handle the last event separately
    if sorted_tide_events:
        last_event = sorted_tide_events[-1]
        if last_tide and last_event['eventDate'] == last_tide['eventDate']:
            # If the last tide is also the last event, it's already classified
            pass
        else:
            # Classify the last event if it's not the last tide
            if len(sorted_tide_events) > 1:
                second_to_last_event = sorted_tide_events[-2]
                if last_event['value'] > second_to_last_event['value']:
                    high_tides.append(last_event)
                else:
                    low_tides.append(last_event)

    # Find the next high tide and next low tide
    high_tide = next((tide for tide in high_tides if dateutil.parser.parse(tide['eventDate']) > current_time), None)
    low_tide = next((tide for tide in low_tides if dateutil.parser.parse(tide['eventDate']) > current_time), None)

    tide_data[station_id] = {
        "station_name": data['station_name'],
        "last_tide": last_tide,
        "high_tide": high_tide,
        "low_tide": low_tide
    }

# Write the relevant tide data to a CSV file
# Write the relevant tide data to a CSV file
# Write the relevant tide data to a CSV file
with open('nowhilo_data.csv', 'w', newline='') as csvfile:
    fieldnames = ['Station ID', 'Station Name', 'Last Tide Date', 'Last Tide Value', 'High Tide Date', 'High Tide Value', 'Low Tide Date', 'Low Tide Value']
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
    writer.writeheader()

    for station_id, tides in tide_data.items():
        row = {
            'Station ID': station_id,
            'Station Name': tides['station_name'],
            'Last Tide Date': tides['last_tide'].get('eventDate') if tides['last_tide'] else '',
            'Last Tide Value': tides['last_tide'].get('value') if tides['last_tide'] else '',
            'High Tide Date': tides['high_tide'].get('eventDate') if tides['high_tide'] else '',
            'High Tide Value': tides['high_tide'].get('value') if tides['high_tide'] else '',
            'Low Tide Date': tides['low_tide'].get('eventDate') if tides['low_tide'] else '',
            'Low Tide Value': tides['low_tide'].get('value') if tides['low_tide'] else ''
        }

        writer.writerow(row)



Failed to retrieve data for station 5cebf1df3d0f4a073c4bbcbb - Status Code: 404


In [6]:
import pandas as pd

# Read the CSV file into a DataFrame
df_nowhilo = pd.read_csv('nowhilo_data.csv')

# Display the DataFrame
#df_nowhilo


In [7]:
from datetime import datetime, timezone

def determine_tide_stage(current_date, last_tide_date, high_tide_date, low_tide_date):
    # Compare current time with the next high and low tides
    time_to_high_tide = (high_tide_date - current_date).total_seconds()
    time_to_low_tide = (low_tide_date - current_date).total_seconds()
    time_from_last_tide = (current_date - last_tide_date).total_seconds()

    # Determine the next tide and calculate the time to/from it
    if time_to_high_tide > 0 and (time_to_high_tide < time_to_low_tide or time_to_low_tide < 0):
        next_tide = 'High Tide'
        time_to_next_tide = time_to_high_tide
    elif time_to_low_tide > 0:
        next_tide = 'Low Tide'
        time_to_next_tide = time_to_low_tide
    else:
        # When both high tide and low tide are in the past, find the next upcoming tide
        if time_to_high_tide > time_to_low_tide:
            next_tide = 'Low Tide'
            time_to_next_tide = time_to_low_tide
        else:
            next_tide = 'High Tide'
            time_to_next_tide = time_to_high_tide

    # Convert seconds to hours
    hours_from_last_tide = time_from_last_tide / 3600
    hours_to_next_tide = time_to_next_tide / 3600 if time_to_next_tide != float('inf') else 0
    return next_tide, hours_to_next_tide, hours_from_last_tide


# Current date and time
current_date = datetime.utcnow().replace(tzinfo=timezone.utc)

# Merge the two dataframes on 'Station ID'
df_combined2 = pd.merge(df_tide, df_nowhilo, on='Station ID')

# Convert date columns to datetime
df_combined2['Prediction Date'] = pd.to_datetime(df_combined2['Event Date'])
df_combined2['Last Tide Date'] = pd.to_datetime(df_combined2['Last Tide Date'])
df_combined2['High Tide Date'] = pd.to_datetime(df_combined2['High Tide Date'])
df_combined2['Low Tide Date'] = pd.to_datetime(df_combined2['Low Tide Date'])

# Apply the corrected function to your dataframe
df_combined2['Next Tide'], df_combined2['Hours to Next Tide'], df_combined2['Hours from Last Tide'] = zip(*df_combined2.apply(
    lambda row: determine_tide_stage(current_date, row["Last Tide Date"], row['High Tide Date'], row['Low Tide Date']), axis=1))

final_columns = ['Station ID', 'Station Name_x', 'Prediction Date', 'Value', 'Last Tide Date', 'Last Tide Value', 'Low Tide Date', 'Low Tide Value', 'High Tide Date', 'High Tide Value', 'Next Tide', 'Hours to Next Tide', "Hours from Last Tide"]
df_final2 = df_combined2[final_columns]

# Optionally rename 'Station Name_x' back to 'Station Name'
df_final2 = df_final2.rename(columns={'Station Name_x': 'Station Name'})

#df_final2



In [8]:
import folium
from folium import IFrame

# Create a base map
m = folium.Map(location=[45, -63], zoom_start=7)

# Function to convert decimal hours to HH:MM format
def format_hours_to_hh_mm(hours):
    total_minutes = int(hours * 60)
    hh = total_minutes // 60
    mm = total_minutes % 60
    return f"{hh:02d}:{mm:02d}"

# Function to calculate total tide time
def tide_cycle_duration(hours_to_next_tide, hours_from_last_tide):
    return hours_to_next_tide + hours_from_last_tide

def get_marker_size(hours_to_next_tide, next_tide, MIN_MARKER_SIZE, MAX_MARKER_SIZE):
    # Calculate the time between the last tide and the next tide
    tide_duration = tide_cycle_duration(row['Hours to Next Tide'], row['Hours from Last Tide'])

    if next_tide == 'High Tide':
        # Tide is rising: Size increases as we approach high tide
        size_factor = 1 - (hours_to_next_tide / tide_duration)
    elif next_tide == 'Low Tide':
        # Tide is falling: Size decreases as we approach low tide
        size_factor = hours_to_next_tide / tide_duration
    else:
        # Default size factor if tide stage is unknown
        size_factor = 0.0

    # Calculate the marker size
    size = MIN_MARKER_SIZE + (MAX_MARKER_SIZE - MIN_MARKER_SIZE) * size_factor
    return size

# Add markers
for index, row in df_final2.iterrows():
    station_id = row['Station ID']
    coords = station_coordinates.get(station_id)

    if not coords or pd.isna(row['Hours to Next Tide']) or pd.isna(row['Hours from Last Tide']):
    # Skip this station if coordinates are missing or any tide data is NaN
        continue

    if coords:
        tide_cycle = tide_cycle_duration(row['Hours to Next Tide'], row['Hours from Last Tide'])
        color = 'blue' if row['Next Tide'] == 'Low Tide' else 'red'


        MAX_MARKER_SIZE = (tide_cycle)*3  # Maximum size for the dynamic marker at high tide
        MIN_MARKER_SIZE = (tide_cycle/5)

        size = get_marker_size(row['Hours to Next Tide'], row['Next Tide'], MIN_MARKER_SIZE, MAX_MARKER_SIZE)

        time_to_next_tide_formatted = format_hours_to_hh_mm(row['Hours to Next Tide'])
        tide_from_last_tide_formatted = format_hours_to_hh_mm(row['Hours from Last Tide'])
        tide_cycle_time_formatted = format_hours_to_hh_mm(tide_cycle)
        next_tide_formatted = row['Next Tide']
        station_name_formatted = row['Station Name']

        popup_content = f"""
        <div style="width: 250px; height: 150; font-size: 12px;">
            <b>Station:</b> {station_name_formatted}<br>
            <b>Next Tide:</b> {next_tide_formatted}<br>
            <b>Hours to Next Tide:</b> {time_to_next_tide_formatted}<br>
            <b>Hours from Last Tide:<b/> {tide_from_last_tide_formatted}<br>
            <b>Tide Cycle Time:<b/> {tide_cycle_time_formatted}
        </div>
        """
        iframe = IFrame(html=popup_content, width=200, height=150)
        popup = folium.Popup(iframe, max_width=250)

        # Draw the dynamic circle
        folium.CircleMarker(
            location=[coords['lat'], coords['lon']],
            radius=size,
            color=color,
            fill=True,
            fill_color=color,
            fill_opacity=1,
            popup=popup
        ).add_to(m)

        # Draw the outline circle
        folium.CircleMarker(
            location=[coords['lat'], coords['lon']],
            radius=MAX_MARKER_SIZE,
            weight=1,
            color='black',
            fill=False
        ).add_to(m)

        # Draw the outline circle
        folium.CircleMarker(
            location=[coords['lat'], coords['lon']],
            radius=MIN_MARKER_SIZE,
            weight=1,
            color='white',
            fill=False
        ).add_to(m)

# Display the map
m
